In [1]:
#####load quanztized_resnet18
import sys
import time
import random
import os
sys.path.append("..")
from module.layer_fusion_module import *
from module.resnet_module import *
from module.extract_weight_module import *
model_dir = "saved_models"
model_filename = "q_resnet18_cifar10.pt"
quantized_model_edited_filename = "q_resnet18_quantized_cifar10_edited.pt"
fusioned_model_filename = "q_fusioned_resnet18_cifar10.pt"
quantized_model_filename_jit = "q_resnet18_quantized_cifar10.pt"
quantized_model_filename = "q_resnet18_quantized_cifar10.pt"

model_filepath = os.path.join(model_dir, model_filename)
quantized_model_filepath = os.path.join(model_dir, quantized_model_filename)
quantized_model_filepath_jit = os.path.join(model_dir, quantized_model_filename_jit)
quantized_model_edited_filepath = os.path.join(model_dir, quantized_model_edited_filename)

num_classes = 10
cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")

model = create_model(num_classes=num_classes)
model.to(cpu_device)

quantized_model = QuantizedResNet18(model_fp32=model)

quantization_config = torch.quantization.get_default_qconfig("fbgemm")

quantized_model.qconfig = quantization_config
fuse_model(quantized_model)
torch.quantization.prepare_qat(quantized_model, inplace=True)
quantized_model.to(cpu_device)    
quantized_model = torch.quantization.convert(quantized_model, inplace=True)
#save_torchscript_model(model=quantized_model, model_dir=model_dir, model_filename=quantized_model_filename_jit)
#save_model(model=quantized_model, model_dir=model_dir, model_filename=quantized_model_filename)
loaded_model=load_model(model=quantized_model, model_filepath=quantized_model_edited_filepath, device='cpu')
loaded_model.eval()
#save each layer weights


{'model.model_fp32.conv1': [['0', '1', '2']], 'model.model_fp32.conv2_x[0].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv2_x[1].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv3_x[0].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv3_x[0].shortcut': [['0', '1', '2']], 'model.model_fp32.conv3_x[1].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv4_x[0].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv4_x[0].shortcut': [['0', '1', '2']], 'model.model_fp32.conv4_x[1].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv5_x[0].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv5_x[0].shortcut': [['0', '1', '2']], 'model.model_fp32.conv5_x[1].residual_function': [['0', '1', '2'], ['3', '4', '5']]}


C:\Users\win0\Anaconda3\envs\pytorch\lib\site-packages\torch\ao\quantization\observer.py:174: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."
C:\Users\win0\Anaconda3\envs\pytorch\lib\site-packages\torch\ao\quantization\observer.py:1109: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  Returning default scale and zero point "


QuantizedResNet18(
  (quant): Quantize(scale=tensor([0.0374]), zero_point=tensor([57]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (model_fp32): ResNet(
    (conv1): Sequential(
      (0): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.05000000074505806, zero_point=0, padding=(3, 3))
      (1): Identity()
      (2): Identity()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (conv2_x): Sequential(
      (0): q_BasicBlock(
        (residual_function): Sequential(
          (0): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.0312686413526535, zero_point=0, padding=(1, 1))
          (1): Identity()
          (2): Identity()
          (3): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.05000000074505806, zero_point=0, padding=(1, 1))
          (4): Identity()
          (5): Identity()
        )
        (shortcut): Sequential()
        (q_func): QFunctional(
          s

In [18]:
input_file_name='0001'
mean_0=0.485
mean_1=0.456
mean_2=0.406
var_0=0.229
var_1=0.224
var_2=0.225
input_h=32
input_w=32
input_c=3
input_fp_np= np.zeros((1,input_c,input_w,input_h), dtype=np.float32)
input_uint8=np.fromfile(input_file_name, dtype=np.uint8)
for h in range(0,input_h):
    for w in range(0,input_w):
        for c in range(0,input_c):
            if c==0:
                input_fp_np[0][c][h][w]=((input_uint8[c+h*input_w*input_c+w*input_c]/255)-mean_0)/var_0
            if c==1:
                input_fp_np[0][c][h][w]=((input_uint8[c+h*input_w*input_c+w*input_c]/255)-mean_1)/var_1
            if c==2:
                input_fp_np[0][c][h][w]=((input_uint8[c+h*input_w*input_c+w*input_c]/255)-mean_2)/var_2
input_fp_np_tensor=torch.from_numpy(input_fp_np)

In [19]:
#get individual output
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [20]:
quantized_model.model_fp32.fc.register_forward_hook(get_activation('fc'))
quantized_model.quant.register_forward_hook(get_activation('quant'))
quantized_model.model_fp32.conv1[0].register_forward_hook(get_activation('conv1'))
quantized_model.model_fp32.conv1[3].register_forward_hook(get_activation('maxpool'))
output = quantized_model(input_fp_np_tensor)

In [61]:
input_fp_np.flatten()[39]

-0.4225533

In [23]:
activation['quant'].int_repr()[0][0][0]

tensor([42, 43, 43, 43, 44, 45, 46, 46, 47, 48, 48, 49, 50, 50, 50, 51, 51, 52,
        50, 51, 55, 53, 51, 53, 53, 52, 51, 51, 51, 51, 48, 46],
       dtype=torch.uint8)

In [32]:
activation['fc'].int_repr()

tensor([[55,  0,  0,  0,  0,  9,  5,  0,  0,  0]], dtype=torch.uint8)

In [24]:
quantized_input=np.fromfile('0001_quantized_input', dtype=np.uint8)

In [42]:
#################quantized input difference
quantized_input=np.fromfile('0001_quantized_input', dtype=np.uint8)
quant_flatten=activation['quant'].int_repr().flatten()
for i in range(0, len(quantized_input)):
    if(quantized_input[i]!=quant_flatten[i]):
        print("quantized_input[",i,"]",quantized_input[i])
        print("quant_flatten[",i,"]",quant_flatten[i])
               
        

quantized_input[ 27 ] 52
quant_flatten[ 27 ] tensor(51, dtype=torch.uint8)
quantized_input[ 39 ] 47
quant_flatten[ 39 ] tensor(46, dtype=torch.uint8)
quantized_input[ 71 ] 46
quant_flatten[ 71 ] tensor(45, dtype=torch.uint8)
quantized_input[ 81 ] 54
quant_flatten[ 81 ] tensor(53, dtype=torch.uint8)
quantized_input[ 83 ] 52
quant_flatten[ 83 ] tensor(51, dtype=torch.uint8)
quantized_input[ 101 ] 45
quant_flatten[ 101 ] tensor(44, dtype=torch.uint8)
quantized_input[ 155 ] 61
quant_flatten[ 155 ] tensor(60, dtype=torch.uint8)
quantized_input[ 167 ] 44
quant_flatten[ 167 ] tensor(43, dtype=torch.uint8)
quantized_input[ 173 ] 44
quant_flatten[ 173 ] tensor(43, dtype=torch.uint8)
quantized_input[ 177 ] 45
quant_flatten[ 177 ] tensor(44, dtype=torch.uint8)
quantized_input[ 205 ] 43
quant_flatten[ 205 ] tensor(42, dtype=torch.uint8)
quantized_input[ 209 ] 47
quant_flatten[ 209 ] tensor(46, dtype=torch.uint8)
quantized_input[ 237 ] 43
quant_flatten[ 237 ] tensor(42, dtype=torch.uint8)
quantized

In [39]:
zxc=quantized_model.quant.scale.numpy()

In [40]:
zxc

array([0.03739064], dtype=float32)